In [2]:
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import os

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [4]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

index_name = "rfp-genie"
namespace = "proposal"

In [5]:
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
spec = ServerlessSpec(cloud="aws", region="us-east-1")
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=1536,
        metric='dotproduct',
        spec=spec
    )

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])



In [10]:
# Load and split the PDF
loader = PyPDFLoader("sample-rfp.pdf")
pages = loader.load_and_split()

In [11]:
pdf_embeddings = [embeddings.aembed_query(page_content) for page_content in pages]

In [12]:
pc.upsert_items(index_name, data=pdf_embeddings)

[<coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfa8f0>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfa960>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfa9d0>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfaa40>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfaab0>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfab20>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfab90>,
 <coroutine object OpenAIEmbeddings.aembed_query at 0x7ec74abfac00>]

In [55]:
from openai import OpenAI
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": ""}
  ]
)
response.choices[0].message.content

'Hello! How can I assist you today?'